In [1]:
import numpy as np


class Martingale:
    def __init__(self, open_price, max_cnt, direction, margin_call, close_call):
        self.max_cnt = max_cnt  # 最大补仓次数
        self.direction = direction  # 方向 buy 多 sell 空
        self.bs = np.around(np.geomspace(1, 256, num=9)
                            ).astype(int)[:max_cnt]  # 补仓数量
        self.margin_call = margin_call  # 跌 | 涨 1% 补仓
        self.close_call = close_call  # 获利多少平仓
        self.curr = 0  # 当前开仓次数

        self.price_lists = np.array([open_price])

        for i in range(1, max_cnt):
            if self.direction == 'buy':
                self.price_lists = np.append(
                    self.price_lists, self.price_lists[-1] - (self.price_lists[-1] * margin_call))
            else:
                self.price_lists = np.append(
                    self.price_lists, self.price_lists[-1] + (self.price_lists[-1] * margin_call))

    def is_add_open(self, price):  # 是否加仓
        if self.direction == 'buy':
            return self.price_lists[self.price_lists > price]
        else:
            return self.price_lists[self.price_lists < price]

    def is_close_all(self, price):  # 是否平仓
        result = 0
        for i in range(0, self.curr):
            result += self.price_lists[i] * self.bs[i]

        avg_price = result / sum(self.bs[:self.curr])

        if self.direction == 'buy':
            return price > avg_price
        else:
            return price < avg_price


mas = Martingale(open_price=2000, max_cnt=5, direction='buy',
                 margin_call=0.01, close_call=0.005)

print(mas.is_add_open(1960))


[2000.  1980.  1960.2]


In [ ]:
import time
from threading import Timer
from huobi.coin_swap.rest import account, market
from config.huobi import ACCESS_KEY, SECRET_KEY

client = market.Market()


def fetchKLines(symbol: str, interval: str, limit: str):
    return client.get_kline(
        {"contract_code": symbol, "period": interval, "size": limit})


def main():
    result = fetchKLines('BTC-USD', '1min', '1')
    print(result)

class RepeatTimer(Timer):
    def run(self):
        while not self.finished.wait(self.interval):
            self.function(*self.args, **self.kwargs)
        self.finished.set()
        print('Done')


timer = RepeatTimer(3, main, [])
timer.start()
